In [3]:
from datetime import datetime

from google.cloud import aiplatform
import os

In [4]:
REGION = "us-central1"
PROJECT_ID = !(gcloud config get-value project)
PROJECT_ID = PROJECT_ID[0]

In [5]:
PROJECT_ID

'qwiklabs-asl-04-76d2f9d7a5ae'

In [6]:
# Set `PATH` to include the directory containing KFP CLI
PATH = %env PATH
%env PATH=/home/jupyter/.local/bin:{PATH}

env: PATH=/home/jupyter/.local/bin:/usr/local/cuda/bin:/opt/conda/bin:/opt/conda/condabin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games:/home/jupyter/.local/bin:


In [7]:
%%bash
ls /home/jupyter/.local/bin

__pycache__
dsl-compile
dsl-compile-v2
identify-cli
jsonschema
kfp
nodeenv
plasma_store
portserver.py
pre-commit
pre-commit-validate-config
pre-commit-validate-manifest
strip-hints
tb-gcp-uploader
tfx


In [8]:
import os

In [9]:
%%bash
cd /home/jupyter/asl-ml-immersion/notebooks/capstone_project/CapStone_Phase_Contrast/src
cat trainer_image_vertex/Dockerfile

FROM us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-8:latest
RUN pip install -U cloudml-hypertune
WORKDIR /app
COPY * /app/

ENTRYPOINT ["python", "task.py"]

In [10]:
IMAGE_NAME = "trainer_image_ct_phase_contrast_vertex"
TAG = "latest"
TRAINING_CONTAINER_IMAGE_URI = f"gcr.io/{PROJECT_ID}/{IMAGE_NAME}:{TAG}"
TRAINING_CONTAINER_IMAGE_URI


'gcr.io/qwiklabs-asl-04-76d2f9d7a5ae/trainer_image_ct_phase_contrast_vertex:latest'

In [11]:

SERVING_CONTAINER_IMAGE_URI="us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-8:latest"

In [12]:
%%bash
ls

ct_phase_contrast_trainer-0.1.tar.gz
ct_phase_contrast_trainer.egg-info
phase_contrast_3d-kamrul.ipynb
phase_contrast_3d.ipynb
phase_contrast_3d_kfp_pipeline.ipynb


In [13]:
os.chdir("/home/jupyter/asl-ml-immersion/notebooks/capstone_project/CapStone_Phase_Contrast/src/")
os.getcwd()

'/home/jupyter/asl-ml-immersion/notebooks/capstone_project/CapStone_Phase_Contrast/src'

In [14]:
# !gcloud builds submit --timeout 15m --tag $TRAINING_CONTAINER_IMAGE_URI trainer_image_vertex

In [15]:
### Compile the pipeline

In [16]:
ARTIFACT_STORE = f"gs://{PROJECT_ID}-kfp-artifact-store"
PIPELINE_ROOT = f"{ARTIFACT_STORE}/pipeline"
DATA_ROOT = f"{ARTIFACT_STORE}/data"

TRAINING_FILE_PATH = f"{DATA_ROOT}/training/dataset.csv"
VALIDATION_FILE_PATH = f"{DATA_ROOT}/validation/dataset.csv"

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BASE_OUTPUT_DIR = f"{ARTIFACT_STORE}/models/{TIMESTAMP}"

%env PIPELINE_ROOT={PIPELINE_ROOT}
%env PROJECT_ID={PROJECT_ID}
%env REGION={REGION}
%env SERVING_CONTAINER_IMAGE_URI={SERVING_CONTAINER_IMAGE_URI}
%env TRAINING_CONTAINER_IMAGE_URI={TRAINING_CONTAINER_IMAGE_URI}
%env TRAINING_FILE_PATH={TRAINING_FILE_PATH}
%env VALIDATION_FILE_PATH={VALIDATION_FILE_PATH}
%env BASE_OUTPUT_DIR={BASE_OUTPUT_DIR}

env: PIPELINE_ROOT=gs://qwiklabs-asl-04-76d2f9d7a5ae-kfp-artifact-store/pipeline
env: PROJECT_ID=qwiklabs-asl-04-76d2f9d7a5ae
env: REGION=us-central1
env: SERVING_CONTAINER_IMAGE_URI=us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-8:latest
env: TRAINING_CONTAINER_IMAGE_URI=gcr.io/qwiklabs-asl-04-76d2f9d7a5ae/trainer_image_ct_phase_contrast_vertex:latest
env: TRAINING_FILE_PATH=gs://qwiklabs-asl-04-76d2f9d7a5ae-kfp-artifact-store/data/training/dataset.csv
env: VALIDATION_FILE_PATH=gs://qwiklabs-asl-04-76d2f9d7a5ae-kfp-artifact-store/data/validation/dataset.csv
env: BASE_OUTPUT_DIR=gs://qwiklabs-asl-04-76d2f9d7a5ae-kfp-artifact-store/models/20230330174953


Let us make sure that the `ARTIFACT_STORE` has been created, and let us create it if not:

In [17]:
!gsutil ls | grep ^{ARTIFACT_STORE}/$ || gsutil mb -l {REGION} {ARTIFACT_STORE}

gs://qwiklabs-asl-04-76d2f9d7a5ae-kfp-artifact-store/


#### Use the CLI compiler to compile the pipeline

In [24]:
PIPELINE_JSON = "ct_phase_contrast_kfp_pipeline.json"

In [25]:
!cat pipeline_vertex/pipeline_prebuilt.py

# Copyright 2021 Google LLC

# Licensed under the Apache License, Version 2.0 (the "License"); you may not
# use this file except in compliance with the License. You may obtain a copy of
# the License at

# https://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS"
# BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either
# express or implied. See the License for the specific language governing
# permissions and limitations under the License.
"""Kubeflow Covertype Pipeline."""
import os

from google.cloud.aiplatform import hyperparameter_tuning as hpt
from google_cloud_pipeline_components.aiplatform import (
    EndpointCreateOp,
    ModelDeployOp,
    ModelUploadOp,
)
from google_cloud_pipeline_components.experimental import (
    hyperparameter_tuning_job,
)
from google_cloud_pipeline_components.experimental.custom_job import (
    CustomTrainingJobOp,
)
from kfp.v

In [26]:
!dsl-compile-v2 --py pipeline_vertex/pipeline_prebuilt.py --output $PIPELINE_JSON

/home/jupyter/.local/lib/python3.7/site-packages/kfp/v2/compiler/compiler.py:1293: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  category=FutureWarning,


**Note:** You can also use the Python SDK to compile the pipeline:

```python
from kfp.v2 import compiler

compiler.Compiler().compile(
    pipeline_func=create_pipeline, 
    package_path=PIPELINE_JSON,
)

```

In [27]:
!head {PIPELINE_JSON}

{
  "pipelineSpec": {
    "components": {
      "comp-custom-training-job": {
        "executorLabel": "exec-custom-training-job",
        "inputDefinitions": {
          "parameters": {
            "base_output_directory": {
              "type": "STRING"
            },


### Deploy the pipeline package

In [28]:
aiplatform.init(project=PROJECT_ID, location=REGION)

pipeline = aiplatform.PipelineJob(
    display_name="ct_phase_contrast_kfp_pipeline",
    template_path=PIPELINE_JSON,
    enable_caching=True,
)

pipeline.run()

Creating PipelineJob
PipelineJob created. Resource name: projects/146339387885/locations/us-central1/pipelineJobs/ct-phase-contrast-kfp-pipeline-20230330183056
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/146339387885/locations/us-central1/pipelineJobs/ct-phase-contrast-kfp-pipeline-20230330183056')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/ct-phase-contrast-kfp-pipeline-20230330183056?project=146339387885
PipelineJob projects/146339387885/locations/us-central1/pipelineJobs/ct-phase-contrast-kfp-pipeline-20230330183056 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/146339387885/locations/us-central1/pipelineJobs/ct-phase-contrast-kfp-pipeline-20230330183056 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/146339387885/locations/us-central1/pipelineJobs/ct-phase-contrast-kfp-pipeline-20230330183056 current state:
PipelineState.PIPE

RuntimeError: Job failed with:
code: 9
message: "The DAG failed because some tasks failed. The failed tasks are: [hyperparameter-tuning-job].; Job (project_id = qwiklabs-asl-04-76d2f9d7a5ae, job_id = 730212610040922112) is failed due to the above error.; Failed to handle the job: {project_number = 146339387885, job_id = 730212610040922112}"
